### function: swath_unit_vectors
* input: lon, lat grids in SWOT LR ocean height files
* output: $\frac{\partial e}{\partial a}$, $\frac{\partial n}{\partial a}$, $\frac{\partial e}{\partial c}$, $\frac{\partial n}{\partial c}$
* east slope: $s_e = \frac{\partial h}{\partial e}$, <br>
north slope: $s_n = \frac{\partial h}{\partial n}$, <br>
cross-track slope: $s_c = \frac{\partial h}{\partial c}$, <br>
along-track slope: $s_a = \frac{\partial h}{\partial a}$. <br>
* $\begin{bmatrix} s_c \\ s_a \end{bmatrix} = 
\begin{bmatrix} \frac{\partial e}{\partial c} & \frac{\partial n}{\partial c} \\ \frac{\partial e}{\partial a} & \frac{\partial n}{\partial a} \end{bmatrix} 
\begin{bmatrix} s_e \\ s_n \end{bmatrix}
$


Esta función, ``swath_unit_vectors``, se utiliza para calcular vectores unitarios en las direcciones a lo largo de la trayectoria (along-track) y transversales a la trayectoria (cross-track) en un conjunto de datos que representan un área geográfica, como los datos satelitales.

In [ ]:
def swath_unit_vectors(lon, lat):
    """
    compute the unit direction vectors for each cell in a swath

    Parameters
    ----------
    lat : array_like
        2D array of latitude values with shape (na, nc).
    lon : array_like
        2D array of longitude values with shape (na, nc).

    Returns
    -------
    uea : array_like
        2D array of the east component of the init vector in the along-track direction
    una : array_like
        2D array of the north component of the init vector in the along-track direction
    uec : array_like
        2D array of the east component of the init vector in the cross-track
    unc : array_like
        2D array of the north component of the init vector in the cross-track
    """
    d2r = np.pi/180.0
    cost = np.cos(lat*d2r)
    lon = np.unwrap(lon, period = 360)

    dna, dnc = np.gradient(lat,1,1)
    dea, dec = np.gradient(lon,1,1)*cost
    dsa = (dea*dea + dna*dna)**0.5
    uea = dea/dsa
    una = dna/dsa
    dsc = (dec*dec + dnc*dnc)**0.5
    uec = dec/dsc
    unc = dnc/dsc

    return uea, una, uec, unc

### function: model_slope

Este código implementa un modelo que calcula la pendiente (slope) en distintas direcciones (a lo largo de la trayectoria y transversal, así como en las direcciones norte y este) a partir de datos de altimetría del satélite SWOT en resolución baja (L2 LR).

In [ ]:
#    input: SWOT L2 LR file name of one file; id of pass (odd-ascending, even-descending)
#    model slope in the along-track, cross-track, north, east directions
def model_slope(multibeam_expert, pass_id):

    # read one file to get the lon/lat 
    ds_multibeam = xr.open_dataset(multibeam_expert)
    num_lines = ds_multibeam.sizes['num_lines']
    num_pixels = ds_multibeam.sizes['num_pixels']
    # get unit vectors in the along-track an cross-track direction
    [uea, una, uer, unr] = swath_unit_vectors(ds_multibeam.longitude.values, ds_multibeam.latitude.values)
    lon = ds_multibeam["longitude"].values
    lon = np.unwrap(lon, period = 360)
    lat = ds_multibeam["latitude"].values
    # create model cross-track slope
    data = {'longitude': lon.flatten(),
            'latitude': lat.flatten()}
    da = 2000
    dr = 2000
    mdt_grid = "datos/dot.cls21.grd"
    east_slope_grid = "datos/east_32.1.nc"
    north_slope_grid = "datos/north_32.1.nc"
    track_points = pd.DataFrame(data)
    # Use grdtrack to sample the grid along the track
    track_data = pygmt.grdtrack(points=track_points, grid=east_slope_grid,newcolname="east_slope")
    north_data = pygmt.grdtrack(points=track_points, grid=north_slope_grid,newcolname="north_slope")
    mdt_data = pygmt.grdtrack(points=track_points, grid=mdt_grid,newcolname="mdt_model")
    track_data['north_slope'] = north_data['north_slope']
    track_data['mdt_model'] = mdt_data['mdt_model']
    # convert to 2d numpy
    east_model = track_data.east_slope.to_numpy().reshape(num_lines, num_pixels)
    north_model = track_data.north_slope.to_numpy().reshape(num_lines, num_pixels)

    sa_mdt, sr_mdt = np.gradient(track_data.mdt_model.to_numpy().reshape(num_lines, num_pixels), da, dr)
    if pass_id%2==1:
        sa_mdt = -sa_mdt
        sr_mdt = -sr_mdt
    determinant = (uer*una-uea*unr) # all ones
    sn_mdt = -uea*sr_mdt+uer*sa_mdt
    se_mdt = una*sr_mdt-unr*sa_mdt
    sa_mdt *= 1000000
    sr_mdt *= 1000000
    sn_mdt *= 1000000
    se_mdt *= 1000000

    sa_model = uea*east_model+una*north_model + sa_mdt
    sr_model = uer*east_model+unr*north_model + sr_mdt
    
    if pass_id%2==1: # ascending
        sa_model = uea*east_model+una*north_model+sa_mdt
        sr_model = uer*east_model+unr*north_model+sr_mdt
        return sa_model, sr_model, north_model+sn_mdt, east_model+se_mdt

    else: # descending
        sa_model = -1*(uea*east_model+una*north_model+sa_mdt)
        sr_model = -1*(uer*east_model+unr*north_model+sr_mdt)
        return sa_model, sr_model, north_model-sn_mdt, east_model-se_mdt


### function: Process one pass (old version)
output to lon, lat, north slope, east slope, VGG, residual north slope, residual east slope, residual VGG, to a pandas dataframe <br>
here we use mean to get the slope residuals, can update codes to use median which is resilient to outliers<br>

In [ ]:
#    input: SWOT L2 LR file names of the same pass (list); pass_id(integer);
#    output: sa, sr, sn, se
def process_onepass_old(list_fnames, pass_id):

    # read one file to get the lon/lat 
    num_files = len(list_fnames)
    if num_files > 1:
        [sa_model, sr_model, sn_model, se_model] = model_slope(list_fnames[0], pass_id)
    else:
        return

    # read all files
    sa_list = []
    sr_list = []
    ssh_list = []
    swh_list = []
    for i in range(num_files):
        ds_expert = xr.open_dataset(list_fnames[i])
        num_lines = ds_expert.sizes['num_lines']
        num_pixels = ds_expert.sizes['num_pixels']
        distance = ds_expert.cross_track_distance.values
        ssha = ds_expert.ssha_karin_2
        ssha = np.where(ds_expert.ssha_karin_2_qual == 0, ssha, np.nan)
        ssha = np.where(ds_expert.ancillary_surface_classification_flag == 0, ssha, np.nan)
        ssh = ssha + ds_expert.mean_sea_surface_cnescls.values+ds_expert.height_cor_xover
        swh = ds_expert.swh_karin
        swh = np.where(ds_expert.ssha_karin_2_qual == 0, swh, np.nan)
        swh = np.where(swh != 0, swh, np.nan)
        da = 2000
        dr = 2000
        sa, sr = np.gradient(ssh, da, dr)
        sa = sa * 10**6 # along-track ssh slope in microradian
        sr = sr * 10**6 # cross-track ssh slope in microradian
        dsr = np.nanmean(sr - sr_model) #remove offset
        sr = sr - dsr
        sa_list.append(sa)
        sr_list.append(sr)
        swh_list.append(swh)

    # 1/swh as weight
    weight = np.nan_to_num(1/np.array(swh_list), nan=0.0)
    weight[weight<(1/6.0)] = 0 # if SWH>6.0m, do not use the data
    masked_data = np.ma.masked_array(np.array(sa_list), np.isnan(np.array(sa_list)))
    sa_stack = np.ma.average(masked_data, axis=0, weights=weight).filled(np.nan)
    masked_data = np.ma.masked_array(np.array(sr_list), np.isnan(np.array(sr_list)))
    sr_stack = np.ma.average(masked_data, axis=0, weights=weight).filled(np.nan)
    if pass_id%2==1: # ascending
        sa_stack = -sa_stack
        sr_stack = -sr_stack
        saa, sar = np.gradient(sa_stack, da, dr)
        sra, srr = np.gradient(sr_stack, da, dr)
        vgg = -(saa + srr)*9.80665*1000 # Eotvos

        saa_m, sar_m = np.gradient(sa_model, da, dr)
        sra_m, srr_m = np.gradient(sr_model, da, dr)
        vgg_model = -(saa_m + srr_m)*9.80665*1000  # Eotvos
        
    else: #descending
        saa, sar = np.gradient(sa_stack, da, dr)
        sra, srr = np.gradient(sr_stack, da, dr)
        vgg = (saa + srr)*9.80665*1000 # Eotvos

        saa_m, sar_m = np.gradient(sa_model, da, dr)
        sra_m, srr_m = np.gradient(sr_model, da, dr)
        vgg_model = (saa_m + srr_m)*9.80665*1000  # Eotvos
        
    vgg_res = vgg - vgg_model
    # remove model slope first
    masked_data = np.ma.masked_array(sr_stack - sr_model, np.isnan(sr_stack - sr_model))
    sr_diff = np.ma.average(masked_data, axis=0).filled(np.nan)
        
    # apply phasescreen correction
    phasescreen = np.array([sr_diff] * num_lines)
    sr_stack_corrected = sr_stack - phasescreen

    # get unit vectors in the along-track an cross-track direction
    [uea, una, uer, unr] = swath_unit_vectors(ds_expert.longitude.values, ds_expert.latitude.values)
    sn_stack = (-uea*sr_stack_corrected+uer*sa_stack)
    se_stack = (una*sr_stack_corrected-unr*sa_stack)
    if pass_id%2==0: # descending
        sn_stack = -sn_stack
        se_stack = -se_stack
    sn_res = sn_stack - sn_model
    se_res = se_stack - se_model

    # create output pandas dataframe
    out = {'longitude': ds_expert["longitude"].values.flatten(),
            'latitude': ds_expert["latitude"].values.flatten(),
          'sn_stack':sn_stack.flatten(),
          'se_stack':se_stack.flatten(),
          'vgg': vgg.flatten(),
          'sn_res':sn_res.flatten(),
          'se_res':se_res.flatten(),
          'vgg_res': vgg_res.flatten()}

    return out

### function: Process one pass 
output to lon, lat, residual north slope, east slope, VGG to a pandas dataframe <br>
here we use mean to get the slope residuals, can update codes to use median which is resilient to outliers<br>

In [ ]:
#    input: SWOT L2 LR file names of the same pass (list); pass_id(integer);
#    output: sa, sr, sn, se
def process_onepass(list_fnames, pass_id):

    # read one file to get the lon/lat 
    num_files = len(list_fnames)
    if num_files > 1:
        [sa_model, sr_model, sn_model, se_model] = model_slope(list_fnames[0], pass_id)
    else:
        return

    # read all files
    sa_list = []
    sr_list = []
    ssh_list = []
    swh_list = []
    for i in range(num_files):
        ds_expert = xr.open_dataset(list_fnames[i])
        num_lines = ds_expert.sizes['num_lines']
        num_pixels = ds_expert.sizes['num_pixels']
        distance = ds_expert.cross_track_distance.values
        ssha = ds_expert.ssha_karin_2
        ssha = np.where(ds_expert.ssha_karin_2_qual == 0, ssha, np.nan)
        ssha = np.where(ds_expert.ancillary_surface_classification_flag == 0, ssha, np.nan)
        ssha = np.where(ds_expert.distance_to_coast.values > 2000, ssha, np.nan)
        ssh = ssha + ds_expert.mean_sea_surface_cnescls.values+ds_expert.height_cor_xover
        swh = ds_expert.swh_karin
        swh = np.where(ds_expert.ssha_karin_2_qual == 0, swh, np.nan)
        swh = np.where(swh != 0, swh, np.nan)

        da = 2000
        dr = 2000
        sa, sr = np.gradient(ssh, da, dr)
        sa = sa * 10**6 # along-track ssh slope in microradian
        sr = sr * 10**6 # cross-track ssh slope in microradian
        dsr = np.nanmean(sr - sr_model)
        sr = sr - dsr
        sa_list.append(sa)
        sr_list.append(sr)
        swh_list.append(swh)

    # uniform weight
    weight = np.nan_to_num(0*np.array(swh_list)+1, nan=1)
    weight[np.array(swh_list)>6] = 0 # if SWH>6.0m, do not use the data
    masked_data = np.ma.masked_array(np.array(sa_list), np.isnan(np.array(sa_list)))
    sa_stack = np.ma.average(masked_data, axis=0, weights=weight).filled(np.nan)
    masked_data = np.ma.masked_array(np.array(sr_list), np.isnan(np.array(sr_list)))
    sr_stack = np.ma.average(masked_data, axis=0, weights=weight).filled(np.nan)

    if pass_id%2==1: # ascending
        sa_stack = -sa_stack
        sr_stack = -sr_stack
        
    # apply phasescreen correction
    masked_data = np.ma.masked_array(sr_stack - sr_model, np.isnan(sr_stack - sr_model))
    sr_diff = np.ma.average(masked_data, axis=0).filled(np.nan) 
    phasescreen = np.array([sr_diff] * num_lines)
    sr_stack_corrected = sr_stack - phasescreen

    sa_stack_res = sa_stack - sa_model
    sr_stack_corrected_res = sr_stack_corrected - sr_model
    if pass_id%2==1: # ascending
        saa_res, sar_res = np.gradient(sa_stack_res, da, dr)
        sra_res, srr_res = np.gradient(sr_stack_corrected_res, da, dr)
        vgg_res = -(saa_res + srr_res)*9.80665*1000 # Eotvos
    if pass_id%2==0: # descending
        saa_res, sar_res = np.gradient(sa_stack_res, da, dr)
        sra_res, srr_res = np.gradient(sr_stack_corrected_res, da, dr)
        vgg_res = (saa_res + srr_res)*9.80665*1000 # Eotvos

    # get unit vectors in the along-track an cross-track direction
    [uea, una, uer, unr] = swath_unit_vectors(ds_expert.longitude.values, ds_expert.latitude.values)
    sn_stack = (-uea*sr_stack_corrected+uer*sa_stack)
    se_stack = (una*sr_stack_corrected-unr*sa_stack)
    if pass_id%2==0: # descending
        sn_stack = -sn_stack
        se_stack = -se_stack
    sn_res = sn_stack - sn_model
    se_res = se_stack - se_model

    # create output pandas dataframe
    out = {'longitude': ds_expert["longitude"].values.flatten(),
            'latitude': ds_expert["latitude"].values.flatten(),
           'sn_res':sn_res.flatten(),
           'se_res':se_res.flatten(),
           'vgg_res': vgg_res.flatten()}

    return out

### function: Process one pass and also output the rms
output to lon, lat, residual north slope, east slope, VGG, and the L2, L1 norm of VGG, to a pandas dataframe <br>
here we use median to get the slope residuals, can update codes to use mean<br>

In [ ]:
def process_onepass_rms(list_fnames, pass_id):

    # read one file to get the lon/lat 
    num_files = len(list_fnames)
    if num_files > 1:
        [sa_model, sr_model, sn_model, se_model] = model_slope(list_fnames[0], pass_id)
    else:
        return

    # read all files
    sa_list = []
    sr_list = []
    swh_list = []
    for i in range(num_files):
        ds_expert = xr.open_dataset(list_fnames[i])
        num_lines = ds_expert.sizes['num_lines']
        num_pixels = ds_expert.sizes['num_pixels']
        distance = ds_expert.cross_track_distance.values
        ssha = ds_expert.ssha_karin_2
        ssha = np.where(ds_expert.ssha_karin_2_qual == 0, ssha, np.nan)
        ssha = np.where(ds_expert.swh_karin.values < 6, ssha, np.nan)
        ssha = np.where(ds_expert.ancillary_surface_classification_flag == 0, ssha, np.nan)
        ssha = np.where(ds_expert.distance_to_coast.values > 2000, ssha, np.nan)
        ssh = ssha + ds_expert.mean_sea_surface_cnescls.values+ds_expert.height_cor_xover
        swh = ds_expert.swh_karin
        swh = np.where(ds_expert.ssha_karin_2_qual == 0, swh, np.nan)
        swh = np.where(swh != 0, swh, np.nan)

        da = 2000
        dr = 2000
        sa, sr = np.gradient(ssh, da, dr)
        sa = sa * 10**6 # along-track ssh slope in microradian
        sr = sr * 10**6 # cross-track ssh slope in microradian
        dsr = np.nanmean(sr - sr_model)
        sr = sr - dsr
        sa_list.append(sa)
        sr_list.append(sr)
        swh_list.append(swh)

    # uniform weight
    weight = np.nan_to_num(0*np.array(swh_list)+1, nan=1)
    weight[np.array(swh_list)>6] = 0 # if SWH>6.0m, do not use the data
    weight[np.isnan(np.array(sa_list))] = 0 # if sa = nan, do not use the data
    weight[np.isnan(np.array(sr_list))] = 0 # if sr = nan, do not use the data
    masked_data = np.ma.masked_array(np.array(sa_list), np.isnan(np.array(sa_list)))
    # sa_stack = np.ma.average(masked_data, axis=0, weights=weight).filled(np.nan)
    sa_stack = np.ma.median(masked_data, axis=0).filled(np.nan)
    masked_data = np.ma.masked_array(np.array(sr_list), np.isnan(np.array(sr_list)))
    # sr_stack = np.ma.average(masked_data, axis=0, weights=weight).filled(np.nan)
    sr_stack = np.ma.median(masked_data, axis=0).filled(np.nan)

    point_used = np.sum((weight!=0), axis = 0)

    # remove pixels with only a few cycles collected
    sa_stack[point_used<8] = np.nan
    sr_stack[point_used<8] = np.nan

    if pass_id%2==1: # ascending
        sa_stack = -sa_stack
        sr_stack = -sr_stack
        
    # apply phasescreen correction
    masked_data = np.ma.masked_array(sr_stack - sr_model, np.isnan(sr_stack - sr_model))
    sr_diff = np.ma.average(masked_data, axis=0).filled(np.nan) 
    phasescreen = np.array([sr_diff] * num_lines)
    sr_stack_corrected = sr_stack - phasescreen

    sa_stack_res = sa_stack - sa_model
    sr_stack_corrected_res = sr_stack_corrected - sr_model

    # calculate VGG from the stacked slope
    if pass_id%2==1: # ascending
        saa_res, sar_res = np.gradient(sa_stack_res, da, dr)
        sra_res, srr_res = np.gradient(sr_stack_corrected_res, da, dr)
        vgg_res = -(saa_res + srr_res)*9.80665*1000 # Eotvos
    if pass_id%2==0: # descending
        saa_res, sar_res = np.gradient(sa_stack_res, da, dr)
        sra_res, srr_res = np.gradient(sr_stack_corrected_res, da, dr)
        vgg_res = (saa_res + srr_res)*9.80665*1000 # Eotvos

    # get unit vectors in the along-track an cross-track direction
    [uea, una, uer, unr] = swath_unit_vectors(ds_expert.longitude.values, ds_expert.latitude.values)
    sn_stack = (-uea*sr_stack_corrected+uer*sa_stack)
    se_stack = (una*sr_stack_corrected-unr*sa_stack)
    if pass_id%2==0: # descending
        sn_stack = -sn_stack
        se_stack = -se_stack
    sn_res = sn_stack - sn_model
    se_res = se_stack - se_model

    # calculate VGG from each cycle
    vgg_list = []
    for i in range(num_files):
        if pass_id%2==1: # ascending
            saa_res, sar_res = np.gradient(-sa_list[i] - sa_model, da, dr)
            sra_res, srr_res = np.gradient(-sr_list[i] - phasescreen - sr_model, da, dr)
            vgg = -(saa_res + srr_res)*9.80665*1000 # Eotvos
            vgg_list.append(vgg)
        if pass_id%2==0: # descending
            saa_res, sar_res = np.gradient(sa_list[i] - sa_model, da, dr)
            sra_res, srr_res = np.gradient(sr_list[i] - phasescreen - sr_model, da, dr)
            vgg = (saa_res + srr_res)*9.80665*1000 # Eotvos
            vgg_list.append(vgg)


    # calculate rms 
    masked_data = np.ma.masked_array(np.array(vgg_list)-vgg_res, np.isnan(np.array(vgg_list)-vgg_res))
    vgg_rms = np.ma.var(masked_data, axis=0).filled(np.nan)
    vgg_rms = vgg_rms**0.5
    masked_data = np.ma.masked_array(np.abs(np.array(vgg_list)-vgg_res), np.isnan(np.array(vgg_list)-vgg_res))
    vgg_L1 = np.ma.median(masked_data, axis=0).filled(np.nan)


    # create output pandas dataframe
    out = {'longitude': ds_expert["longitude"].values.flatten(),
            'latitude': ds_expert["latitude"].values.flatten(),
           'sn_res':sn_res.flatten(),
           'se_res':se_res.flatten(),
           'vgg_res': vgg_res.flatten(),
           'vgg_rms': vgg_rms.flatten(),
           'vgg_L1': vgg_L1.flatten()}

    return out
